In [1]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import argparse

In [1]:
import gzip
import csv
with gzip.open('/homeb/xiaoyf/data/HG002/R9.4/samples_CG.hc_poses.r30m.tsv.gz', 'rt') as f:
    data = f.read().decode()
    tsv_reader = csv.reader(data, delimiter="\t")

    number_of_lines = 10

    for i in range(number_of_lines):
        row = next(tsv_reader)
        print(i, row)

MemoryError: 

In [1]:
import pandas as pd
chunks = pd.read_csv('/homeb/xiaoyf/data/HG002/R9.4/samples_CG.hc_poses.r30m.tsv',sep='\t',error_bad_lines=False,iterator = True,header=None)
chunk = chunks.get_chunk(5)



/tmp/ipykernel_39182/2179687221.py:2: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  chunks = pd.read_csv('/homeb/xiaoyf/data/HG002/R9.4/samples_CG.hc_poses.r30m.tsv',sep='\t',error_bad_lines=False,iterator = True,header=None)


在deepsignal2里面一个碱基固定对应16个信号

In [32]:
sigs=chunk.iloc[:,-2]
for s in sigs:
    sig=np.array([[float(y) for y in x.split(",")] for x in s.split(";")])
    print(sig.shape)


(17, 16)
(17, 16)
(17, 16)
(17, 16)
(17, 16)


In [2]:
def parse_args():
    parser = argparse.ArgumentParser()
    parser.add_argument(
        "--conv-in", type=int, default=4, help="Input sequence features"
    )
    parser.add_argument("--batch_size", type=int, default=512, required=False)
    parser.add_argument("--step_interval", type=int, default=100, required=False)
    parser.add_argument("--lr", type=float, default=0.001, required=False)
    parser.add_argument(
        "--train-file",
        type=str,
        help="feature file used in trainning",
        default="/homeb/xiaoyf/data/HG002/R9.4/samples_CG.hc_poses.r30m.tsv",
    )
    parser.add_argument(
        "--model_dir", type=str, default="/home/xiaoyf/methylation/deepsignal/log/"
    )
    parser.add_argument(
        "--max_epoch_num",
        action="store",
        default=10,
        type=int,
        required=False,
        help="max epoch num, default 10",
    )
    parser.add_argument(
        "--min_epoch_num",
        action="store",
        default=5,
        type=int,
        required=False,
        help="min epoch num, default 5",
    )
    return parser.parse_args([])

In [3]:
def squash(tensor, dim=-1):
    squared_norm = (tensor**2).sum(dim=dim, keepdim=True)
    scale = squared_norm / (1 + squared_norm)
    return scale * tensor / torch.sqrt(squared_norm)

In [4]:
class Squash(nn.Module):
    def __init__(self, eps=10e-21, **kwargs):
        super(Squash, self).__init__(**kwargs)
        self.eps = eps

    def forward(self, s):
        n = torch.norm(s, dim=-1, keepdim=True)
        return (1 - 1 / (torch.exp(n) + self.eps)) * (s / (n + self.eps))

In [5]:
def dynamic_routing(x, iterations=3):
    #x = x.unsqueeze(-1)
    N = x.shape[1]  # num_caps
    N1 = x.shape[2]  # in_caps
    B = x.shape[0]
    # feature_dim = x.shape[2]
    #x:batch_size, num_caps, in_caps, out_channels
    b = torch.zeros(B, N, N1,1).to(x.device)#batch_size, num_caps, in_caps
    for _ in range(iterations):
        #print('input x\'s batch_size: {}, num_caps: {}, in_caps: {}, out_channels: {}'.format(x.shape[0], x.shape[1], x.shape[2], x.shape[3]))
        c = F.softmax(b, dim=1)#Softmax along num_caps
        #batch_size, num_caps,caps_dim
        #print('softmax result\'s batch_size: {}, num_caps: {}, in_caps: {}, softmax_result: {}'.format(c.shape[0], c.shape[1], c.shape[2], c.shape[3]))
        a = c*x
        #print('a\'s batch_size: {}, num_caps: {}, in_caps: {}, out_channels: {}'.format(a.shape[0], a.shape[1], a.shape[2], a.shape[3]))
        s = torch.sum(a, dim=2).squeeze(-1)#sum across in_caps
        #print('s\'s batch_size: {}, num_caps: {}, out_channels: {}'.format(s.shape[0], s.shape[1], s.shape[2]))
        v = squash(s)# apply "squashing" non-linearity along out_channels
        #print('v\'s batch_size: {}, num_caps: {}, out_channels: {}'.format(v.shape[0], v.shape[1], v.shape[2]))
        #print('x shape: {}'.format(x.shape))
        y = torch.matmul(x,v.unsqueeze(-1))
        #print('y shape: {}'.format(y.shape))
        #print('b shape: {}'.format(b.shape))
        b = b + y

    return v

In [110]:
#x:batch_size, num_caps, in_caps, out_channels
input_tensor=torch.randn(2, 2, 5,205)
print(dynamic_routing(input_tensor).shape)

input x's batch_size: 2, num_caps: 2, in_caps: 5, out_channels: 205
softmax result's batch_size: 2, num_caps: 2, in_caps: 5, softmax_result: 1
a's batch_size: 2, num_caps: 2, in_caps: 5, out_channels: 205
s's batch_size: 2, num_caps: 2, out_channels: 205
v's batch_size: 2, num_caps: 2, out_channels: 205
x shape: torch.Size([2, 2, 5, 205])
y shape: torch.Size([2, 2, 5, 1])
b shape: torch.Size([2, 2, 5, 1])
input x's batch_size: 2, num_caps: 2, in_caps: 5, out_channels: 205
softmax result's batch_size: 2, num_caps: 2, in_caps: 5, softmax_result: 1
a's batch_size: 2, num_caps: 2, in_caps: 5, out_channels: 205
s's batch_size: 2, num_caps: 2, out_channels: 205
v's batch_size: 2, num_caps: 2, out_channels: 205
x shape: torch.Size([2, 2, 5, 205])
y shape: torch.Size([2, 2, 5, 1])
b shape: torch.Size([2, 2, 5, 1])
input x's batch_size: 2, num_caps: 2, in_caps: 5, out_channels: 205
softmax result's batch_size: 2, num_caps: 2, in_caps: 5, softmax_result: 1
a's batch_size: 2, num_caps: 2, in_caps

In [ ]:
import torch.nn as nn

# Define the input and output channels
in_channels = 2
out_channels = 1

# Define the kernel size and dilation
kernel_size = 2

# Define the 1D dilated convolution layers
conv1d_list = nn.ModuleList()
for dilation in range(1, 6):
    padding = "same"
    conv1d_list.append(
        nn.Conv1d(
            in_channels, out_channels, kernel_size, dilation=dilation, padding=padding
        )
    )

# Define the input tensor
input_tensor = torch.randn(1, in_channels, 21 * 5)

# Apply the 1D dilated convolutions to the input tensor
output_tensor_list = []
for conv1d in conv1d_list:
    print(conv1d(input_tensor).shape)
    output_tensor_list.append(conv1d(input_tensor))

# Concatenate the output tensors along the channel dimension
output_tensor = torch.cat(output_tensor_list, dim=1)

print(output_tensor.shape)
print(dynamic_routing(output_tensor).shape)

In [16]:
c = F.softmax(torch.zeros(1, 1, 5, 1, 1), dim=1)
# print(c)
print(c.shape)
x = torch.randn(1, 5, 21 * 5, 1)
a = x.matmul(c)
# print(a)
print(a.shape)
s = torch.sum(x.matmul(c), dim=2).squeeze(-1)
# print(s)
print(s.shape)

torch.Size([1, 1, 5, 1, 1])
torch.Size([1, 1, 5, 105, 1])
torch.Size([1, 1, 105])


In [13]:
class PrimaryCapsuleLayer(nn.Module):
    """
    Create a primary capsule layer with the methodology described in 'Efficient-CapsNet: Capsule Network with Self-Attention Routing'.
    Properties of each capsule s_n are exatracted using a 1D depthwise convolution.

    ...

    Attributes
    ----------
    kernel_size[w]: int
        depthwise conv kernel dimension
    conv_num: int
        number of primary capsules
    feature_dimension: int
        primary capsules dimension (number of properties)
    conv_stride: int
        depthwise conv strides
    Methods
    -------
    forward(inputs)
        compute the primary capsule layer
    """

    def __init__(
        self,
        conv_in=2,
        feature_dimension=272,#21 * 5,
        kernel_size=2,
        conv_num=5,
        base_num=21,
    ):
        super().__init__()
        
        self.conv_out = feature_dimension // (conv_num * base_num)
        self.conv_num = conv_num
        self.primary_capsule_layer = nn.ModuleList(
            [
                nn.Conv1d(
                    conv_in,
                    self.conv_out,
                    kernel_size,
                    dilation=conv_stride,
                    padding="same",
                )
                for conv_stride in range(1, conv_num + 1)
            ]
        )

    def forward(self, x):
        
        #print('input feature shape: {}'.format(x.shape))
        capsules = [conv(x) for conv in self.primary_capsule_layer]
        # capsules_reshaped = [
        #    c.reshape(self.conv_num, self.feature_dimension) for c in capsules
        # ]
        output_tensor = torch.cat(capsules, dim=1)
        return Squash()(output_tensor)


def test_for_primary_capsule_layer():
    input = torch.rand(1, 2, 105)
    layer = PrimaryCapsuleLayer()
    print(layer(input).shape)


test_for_primary_capsule_layer()

torch.Size([1, 10, 105])


In [12]:
class CapsLayer(nn.Module):
    def __init__(
        self, num_capsules=1, in_caps=10, in_channels=272, out_channels=2#in_channels=105
    ):
        super(CapsLayer, self).__init__()
        self.W = nn.Parameter(
            torch.randn(1, num_capsules, in_caps, out_channels, in_channels)
        )
        # print('W shape: {}'.format(self.W.shape))

    def forward(self, x):
        #print('CapsLayer input shape: {}'.format(x.shape))
        x = x[:, None, ..., None]  # x.unsqueeze(1).unsqueeze(4)
        # x = x.unsqueeze(-1)
        #print('W shape: {}'.format(self.W.shape))
        #print('CapsLayer input shape: {}'.format(x.shape))
        # print('CapsLayer input expand shape: {}'.format(x[ :, :, None, :].shape))
        # (batch_size, num_caps, num_route_nodes, out_channels, 1)
        # print('x shape: {}'.format(x.shape))
        u_hat = torch.matmul(self.W, x)  # (x @ self.W).squeeze(2)
        # u=u_hat.squeeze(-1)
        u_hat = u_hat.squeeze(-1)
        #batch_size, num_caps, in_caps, out_channels
        #print('u_hat\'s batch_size: {}, num_caps: {}, in_caps: {}, out_channels: {}'.format(u_hat.shape[0], u_hat.shape[1], u_hat.shape[2], u_hat.shape[3]))
        class_capsules = dynamic_routing(u_hat)
        return class_capsules

In [54]:
a = torch.rand(1, 10, 10, 20, 1)
b = torch.rand(1, 1, 10, 1, 1)
c = a.matmul(b)

In [107]:
input = torch.rand(2, 5, 105)
layer = CapsLayer()
print(layer(input).shape)

CapsLayer input shape: torch.Size([2, 5, 105])
W shape: torch.Size([1, 1, 5, 2, 105])
CapsLayer input shape: torch.Size([2, 1, 5, 105, 1])
u_hat's batch_size: 2, num_caps: 1, in_caps: 5, out_channels: 2
input x's batch_size: 2, num_caps: 1, in_caps: 5, out_channels: 2
softmax result's batch_size: 2, num_caps: 1, in_caps: 5, softmax_result: 1
a's batch_size: 2, num_caps: 1, in_caps: 5, out_channels: 2
s's batch_size: 2, num_caps: 1, out_channels: 2
v's batch_size: 2, num_caps: 1, out_channels: 2
x shape: torch.Size([2, 1, 5, 2])
y shape: torch.Size([2, 1, 5, 1])
b shape: torch.Size([2, 1, 5, 1])
input x's batch_size: 2, num_caps: 1, in_caps: 5, out_channels: 2
softmax result's batch_size: 2, num_caps: 1, in_caps: 5, softmax_result: 1
a's batch_size: 2, num_caps: 1, in_caps: 5, out_channels: 2
s's batch_size: 2, num_caps: 1, out_channels: 2
v's batch_size: 2, num_caps: 1, out_channels: 2
x shape: torch.Size([2, 1, 5, 2])
y shape: torch.Size([2, 1, 5, 1])
b shape: torch.Size([2, 1, 5, 1])

(512,1,272)

In [8]:
class CapsNet(nn.Module):
    def __init__(self,vocab_size=16,embedding_size=16):
        super(CapsNet, self).__init__()
        self.embed = nn.Embedding(vocab_size, embedding_size)
        self.primary_layer = PrimaryCapsuleLayer()
        self.caps_layer = CapsLayer()
        self.softmax = nn.Softmax(1)

    def forward(self, seq,sig):
        seq_emb = self.embed(seq.long())
        seq_emb = seq_emb.reshape(seq_emb.shape[0], 1, -1)
        sig = sig.reshape(sig.shape[0], 1, -1)
        print('seq_emb shape: {}'.format(seq_emb.shape))
        print('sig shape: {}'.format(sig.shape))
        x = torch.cat((seq_emb,sig), dim=1).to(torch.float32)
        #seq = self.primary_layer(seq)
        #seq = self.caps_layer(seq)
        #sig = self.primary_layer(sig)
        #sig = self.caps_layer(sig)
        x = self.primary_layer(x)
        x = self.caps_layer(x)
        #print(x.shape)
        x = torch.norm(x,dim=1)
        return x


def test_for_caps_net():
    input1 = torch.rand(1, 1, 105)
    input2 = torch.rand(1, 1, 105)
    
    model = CapsNet()
    #print(model(input1,input2).shape)


test_for_caps_net()

In [9]:
class CapsuleLoss(nn.Module):
    def __init__(self):
        super(CapsuleLoss, self).__init__()
        self.loss = nn.CrossEntropyLoss()

    def forward(self, classes, labels):
        #classes = classes.reshape(classes.shape[0], 2)
        #labels = labels.reshape(labels.shape[0], 1)
        #print('classes shape: {}'.format(classes.shape))
        #print('labels shape: {}'.format(labels.shape))
        #left = F.relu(0.9 - classes[0], inplace=True) ** 2
        #print('left shape: {}'.format(left.shape))
        #right = F.relu(classes[1] - 0.1, inplace=True) ** 2
        #print('right shape: {}'.format(right.shape))

        #margin_loss = labels * left + 0.5 * (1.0 - labels) * right
        #margin_loss = margin_loss.sum()
        return self.loss(classes, labels)

In [17]:
np.random.randint(0, 2)

1

In [10]:
from torch.utils.data import Dataset
import linecache

base2code_dna = {"A": 0, "C": 1, "G": 2, "T": 3, "N": 4}
code2base_dna = {0: "A", 1: "C", 2: "G", 3: "T", 4: "N"}


def clear_linecache():
    # linecache should be treated carefully
    linecache.clearcache()


def parse_a_line(line):
    words = line.strip().split("\t")

    seq = np.array(
        [[base2code_dna[y] for y in x.split(",")] for x in words[1].split(";")]
    )
    signal = np.array(
        [[np.float16(y) for y in x.split(",")] for x in words[2].split(";")]
    )
    label = np.random.randint(0, 2)
    #if rlabel==1:
    #    label=[0,1]
    #else:
    #    label=[1,0]

    return seq, signal, label


class SignalDataset(Dataset):
    def __init__(self, filename, transform=None):
        # print(">>>using linecache to access '{}'<<<\n"
        #       ">>>after done using the file, "
        #       "remember to use linecache.clearcache() to clear cache for safety<<<".format(filename))
        self._filename = os.path.abspath(filename)
        self._total_data = 0
        self._transform = transform
        with open(filename, "r") as f:
            self._total_data = len(f.readlines())

    def __getitem__(self, idx):
        line = linecache.getline(self._filename, idx + 1)
        if line == "":
            return None
        else:
            output = parse_a_line(line)
            if self._transform is not None:
                output = self._transform(output)
            return output

    def __len__(self):
        return self._total_data

def parse_a_line2(line):
    words = line.strip().split("\t")

    sampleinfo = "\t".join(words[0:6])

    kmer = np.array([base2code_dna[x] for x in words[6]])
    base_means = np.array([float(x) for x in words[7].split(",")])
    base_stds = np.array([float(x) for x in words[8].split(",")])
    base_signal_lens = np.array([int(x) for x in words[9].split(",")])
    k_signals = np.array([[float(y) for y in x.split(",")] for x in words[10].split(";")])
    label = int(words[11])

    return kmer, k_signals, label   
class SignalFeaData2(Dataset):
    def __init__(self, filename, transform=None):
        # print(">>>using linecache to access '{}'<<<\n"
        #       ">>>after done using the file, "
        #       "remember to use linecache.clearcache() to clear cache for safety<<<".format(filename))
        self._filename = os.path.abspath(filename)
        self._total_data = 0
        self._transform = transform
        with open(filename, "r") as f:
            self._total_data = len(f.readlines())

    def __getitem__(self, idx):
        line = linecache.getline(self._filename, idx + 1)
        if line == "":
            return None
        else:
            output = parse_a_line2(line)
            if self._transform is not None:
                output = self._transform(output)
            return output

    def __len__(self):
        return self._total_data


In [ ]:
import sys
from torch.optim.lr_scheduler import StepLR
import time
from sklearn import metrics
import re
from tqdm import tqdm


from torch.utils.data import sampler
import warnings
warnings.simplefilter('ignore')

if __name__ == "__main__":
    total_start = time.time()
    args = parse_args()
    train_dataset = SignalFeaData2(args.train_file)

    split_num = int(len(train_dataset) * 0.8)
    index_list = list(range(len(train_dataset)))
    train_idx, val_idx = index_list[:split_num], index_list[split_num:]
 
    train_sampler = sampler.SubsetRandomSampler(train_idx)
    val_sampler = sampler.SubsetRandomSampler(val_idx)

    train_loader = torch.utils.data.DataLoader(
        dataset=train_dataset, batch_size=args.batch_size, sampler=train_sampler
    )
    total_step = len(train_loader)
    valid_loader = torch.utils.data.DataLoader(
        dataset=train_dataset, batch_size=args.batch_size, sampler=val_sampler
    )
    model = CapsNet()
    criterion = CapsuleLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=args.lr)
    scheduler = StepLR(optimizer, step_size=2, gamma=0.1)
    curr_best_accuracy = 0
    model_dir = args.model_dir
    if model_dir != "/":
        model_dir = os.path.abspath(model_dir).rstrip("/")
        if not os.path.exists(model_dir):
            os.makedirs(model_dir)
        else:
            model_regex = re.compile(
                r"" + "\.b\d+_s\d+_epoch\d+\.ckpt*"
            )
            for mfile in os.listdir(model_dir):
                if model_regex.match(mfile):
                    os.remove(model_dir + "/" + mfile)
        model_dir += "/"
    model.train()
    for epoch in range(args.max_epoch_num):
        curr_best_accuracy_epoch = 0
        no_best_model = True
        tlosses = []
        start = time.time()
        for i, sfeatures in tqdm(enumerate(train_loader)):
            (seq, signal, labels) = sfeatures
            outputs = model(seq, signal)
            loss = criterion(outputs, labels)
            #print(loss)
            print(loss.detach().item())
            tlosses.append(loss.detach().item())

            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
            optimizer.step()
            if (i + 1) % args.step_interval == 0 or i == total_step - 1:
                model.eval()
                with torch.no_grad():
                    vlosses, vlabels_total, vpredicted_total = [], [], []
                    for vi, vsfeatures in tqdm(enumerate(valid_loader)):
                        (
                            vseq,
                            vsignal,
                            vlabels,
                        ) = vsfeatures
                        voutputs = model(vseq, vsignal)
                        vloss = criterion(voutputs, vlabels)

                        _, vpredicted = torch.max(voutputs.data, 1)
                        #print(vpredicted)
                        vlosses.append(vloss.item())
                        vlabels_total += vlabels.tolist()
                        vpredicted_total += vpredicted.tolist()
                        v_accuracy = metrics.accuracy_score(
                            vlabels_total, vpredicted_total
                        )
                        v_precision = metrics.precision_score(
                            vlabels_total, vpredicted_total
                        )
                        v_recall = metrics.recall_score(vlabels_total, vpredicted_total)
                        if v_accuracy > curr_best_accuracy_epoch:
                            curr_best_accuracy_epoch = v_accuracy
                            if curr_best_accuracy_epoch > curr_best_accuracy - 0.0002:
                                torch.save(
                                    model.state_dict(),
                                    model_dir
                                    + ".epoch{}.ckpt".format(
                                        epoch
                                    ),
                                )
                                if curr_best_accuracy_epoch > curr_best_accuracy:
                                    curr_best_accuracy = curr_best_accuracy_epoch
                                    no_best_model = False
                        time_cost = time.time() - start
                        print(
                            "Epoch [{}/{}], Step [{}/{}], TrainLoss: {:.4f}; "
                            "ValidLoss: {:.4f}, "
                            "Accuracy: {:.4f}, Precision: {:.4f}, Recall: {:.4f}, "
                            "curr_epoch_best_accuracy: {:.4f}; Time: {:.2f}s".format(
                                epoch + 1,
                                args.max_epoch_num,
                                i + 1,
                                total_step,
                                np.mean(tlosses),
                                np.mean(vlosses),
                                v_accuracy,
                                v_precision,
                                v_recall,
                                curr_best_accuracy_epoch,
                                time_cost,
                            )
                        )
                        tlosses = []
                        start = time.time()
                        sys.stdout.flush()
                    model.train()
            scheduler.step()
            if no_best_model and epoch >= args.min_epoch_num - 1:
                print("early stop!")
                break
        endtime = time.time()
        clear_linecache()
        print(
            "[main] train costs {} seconds, "
            "best accuracy: {}".format(endtime - total_start, curr_best_accuracy)
        )